In [35]:
import pandas as pd
from bokeh.plotting import figure, output_file, show, save
from bokeh.models import ColumnDataSource, LabelSet
from bokeh.models import BasicTickFormatter, Label, Title, NumeralTickFormatter
# HoverTool allows you to set a tooltips property which takes a list of tuples
from bokeh.models.tools import HoverTool
# a pre-made five color pallette, one of Bokeh’s many pre-made color palettes
from bokeh.palettes import TolRainbow, Category20c, Spectral6
# a helper method for mapping colors to bars in a bar-charts
from bokeh.transform import factor_cmap
import math

datapath = 'datapandas/'
inputFile = 'ABS2021StatePopulation.xlsx'     # population count by State by AgeGroup by Sex

# Function "output_file" defines the output name and how the visualization will be rendered by bokeh, eg html for this one.
# File will be saved in the main project folder
output_file('codeflask/templates/populationByAgegrpBySex.html')

# Read input excel file
dfPopAgegrpBySex = pd.read_excel(datapath+inputFile)
#print(dfPopAgegrpBySex,'\n')

# Group population data by Desc
dfGrouped = dfPopAgegrpBySex.groupby('Desc')[['Population']].sum().reset_index()
#print(dfGrouped,'\n')

# Re-order data, move rows 4 and 5 to the bottom of dataframe
df1 = dfGrouped.drop([4,5])
df2 = dfGrouped.iloc[4:6]
dfGrouped=pd.concat([df1,df2], ignore_index = True)  # Concatenate dataframes
#print(dfGrouped,'\n')

# Add 'colors' column at the end using colors from palette 'TolRainbow'
dfGrouped['colors'] =  ['red', 'blue', 'red', 'blue', 'red', 'blue', 'red', 'blue', 'red', 'blue',
                        'red', 'blue', 'red', 'blue', 'red', 'blue', 'red', 'blue', 'red', 'blue',
                        'red', 'blue']

# Set dataframe into 'column data source' so it can be processed by bokeh
source = ColumnDataSource(dfGrouped)

# Get list of Categorical 'Description' values for vertical bar chart (x-axis)
ageDesc = source.data['Desc'].tolist()
#print(ageDesc,'\n')

# Create 'figure' object, the core object that handles the styling of plots in bokeh.
# Pass ageDesc to x_range in the 'figure' constructor
p = figure(x_range=ageDesc, width=700, height=400
           , background_fill_color="#292929" #404040
           , border_fill_color = "black"
           , outline_line_color= "white"
           , outline_line_width = 0
           )

# Plot data as individually colored bars and add basic labels.
# To color the bars we use the factor_cmap helper function.
# This creates a special color map that matches an individual color to each Category (ageDesc)
color_map = factor_cmap(field_name='Desc', # create a special color map that matches an individual color to each category
                        palette=Spectral6,
                        factors=ageDesc)      # factors in bokeh refers to Categories.
# Call vbar method to create a vertical bar Glyph
p.vbar(x='Desc', top='Population', source=source, width=0.50, fill_color='colors') #color=color_map)
# Add labels
p.title.text ='Population by Age Group by Male,Female as at 2021 Census'
p.xaxis.axis_label = 'Age Group by Male,Female'
p.yaxis.axis_label = 'Population'

#------------------------------------------------------------------------
# Specify Axis labels
##axis_label_text_color = 'red'
##axis_line_color = 'red'
p.yaxis.major_label_text_color = "#e0e0e0"        #ok
p.yaxis.axis_label_text_color = "#e0e0e0"         #ok
p.xaxis.major_label_text_color = "#e0e0e0"        #ok
p.yaxis.major_label_text_font_size = "9pt"
p.xaxis.major_label_text_font_size = "8pt"
p.xaxis.axis_label_text_color = "#e0e0e0"         #ok

# Change orientation of x-axis labels
p.xaxis.major_label_orientation = math.pi/3
#p.xaxis.major_label_orientation = "vertical"

# Change scientific notation in y-axis
p.yaxis[0].formatter = NumeralTickFormatter(format="0,0")

# Chart title
p.title.text_color = "#e0e0e0"
#Hide Gridlines
p.xgrid.grid_line_color = "#e0e0e0"
p.ygrid.grid_line_color = "#e0e0e0"
#--------------------------------------------------------------------------
# Add hover
hover = HoverTool()    # HoverTool allows you to set a tooltips property which takes a list of tuples
hover.tooltips = [("","@Desc; Population:@Population{0,0}")]
hover.mode = 'vline'   # tell the popup to show when a vertical line crosses a glyph
p.add_tools(hover)     # add hover tool in the toolbar.

show(p)
#save(p)

#end of file